# Kwere Character-Level Language Model Using PyTorch LSTMs

This notebook is nearly identical to "Anything Goes (Swahili)". The pretrain-train relationship is just inverted: pre-train on Swahili, fine-tune on Kwere.

### Results

I reached a cross entropy of approximately 1.4 for validation and 1.47 for test. This accuracy was about 67% for the test set.

A lot of variable decisions were surprising, for example: `carry_hidden_state` reduced performance and any more than a very small subset of the Swahili dataset would reduce performance (possibly because it would overwhelm the small amount of Kwere data).

[Click here for a full list of trials run](https://ui.neptune.ai/gregrolwes/Bantu-Language-Modeling/experiments?viewId=standard-view&sortBy=%5B%22timeOfCreation%22%5D&sortDirection=%5B%22descending%22%5D&sortFieldType=%5B%22native%22%5D&sortFieldAggregationMode=%5B%22auto%22%5D&trashed=false&suggestionsEnabled=true&tags=%5B%22kwere%22%2C%22anything%20goes%22%5D&lbViewUnpacked=true)

### Parameters
Dictionary containing all parameters for ease of tuning. These will be logged to the neptune logger below.

**To add test data, enter the test file name in the `test_data` parameter.**

Select Parameter Descriptions:
 - `experiment_name`: identifier to be used in logging
 - `tags`: also for logging and filtering trials
 - `seq_len`: length of character lists fed to the model
 - `num_layers`: LSTM layers
 - `carry_hidden_state`: whether or not to perpetuate the hidden state between sequences
 - `pretrain_lr`: the learning rate to use while pretraining
 - `swahili_percentage`: the percentage of the Swahili data to pretrain with

In [79]:
PARAMS = {
    'experiment_name': "Kwere",
    'tags': ["kwere", "swahili-pretrained", "anything goes"],
    'epochs': 10,
    'hidden_size': 512,
    'seq_len': 500,
    'num_layers': 4,
    'dropout': 0.3,
    'lr': 0.001,
    'carry_hidden_state': False,
    'val_split': 0.3,
    'kwere_train': "./cwe-train.txt",
    'pretrain_epochs': 5,
    'pretrain_lr': 0.0001,
    'swahili_percentage': 0.01, 
    'swahili': "./sw-train.txt",
    'test_data': "./cwe-test.txt"
}

### Logging
For this project I used a logging library / UI called [Neptune.ai](https://neptune.ai/) to track all runs and their respective hyperparameters. Since the API key for this is only in my local `bash_profile`, **this cell will throw an error**, but I'll conditionalize all the logging in the notebook so errors won't be thrown beyond this cell.

To view the logs from my runs, visit the project url [here](https://ui.neptune.ai/gregrolwes/Bantu-Language-Modeling/experiments?viewId=standard-view&sortBy=%5B%22timeOfCreation%22%5D&sortDirection=%5B%22descending%22%5D&sortFieldType=%5B%22native%22%5D&sortFieldAggregationMode=%5B%22auto%22%5D&trashed=false&suggestionsEnabled=true&tags=%5B%22kwere%22%2C%22anything%20goes%22%5D&lbViewUnpacked=true). Note projects are tagged with which language they're targeting, and whether or not they using the "Anything Goes" or "From Scratch" implementation.

In [80]:
is_logging = False

import neptune

neptune.init('gregrolwes/Bantu-Language-Modeling')

neptune.create_experiment(
            name=PARAMS['experiment_name'],
            tags=PARAMS['tags'],
            params=PARAMS
        )

# reach this if the above logger initialization passes
is_logging = True

https://ui.neptune.ai/gregrolwes/Bantu-Language-Modeling/e/BAN-33


### Imports

In [81]:
import torch
import torch.nn as nn
import math

### Random Seed
Make the experiment reproducible.

In [82]:
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### GPU Support

In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Dataset Class
The `Dataset` class generates a list of all unique characters found in the supplied data, number of total characters, number of unique characters, mappings from characters to their respective ID, mappings from chracter IDs to characters for making outputs readable, and a data tensor of every character converted to its ID.

The `Dataset` will also generate a `~` character to be used in place of any characters unknown to the model (i.e. anything not in the training set). See the `clean_data` function below.

Inputs:
 - `raw_data`: `string` of all characters from the provided data in order
 - `device`: `torch.device` of either `cuda` or `cpu`

In [84]:
class Dataset():
    def __init__(self, raw_data: str, device: torch.device):
        self.chars = set(list(set(raw_data)))
        self.chars.add('~')
        self.data_size, self.vocab_size = len(raw_data), len(self.chars)
        print("{} characters, {} unique".format(self.data_size, self.vocab_size))
        
        self.char_to_idx = { char: idx for idx, char in enumerate(self.chars) }
        self.idx_to_char = { idx: char for idx, char in enumerate(self.chars) }
        
        self.data = torch.tensor([self.char_to_idx[char] for char in list(raw_data)]).unsqueeze(1).to(device)
    
    def __len__(self):
        return self.data_size
    
    def __getitem__(self, index):
        return self.data[index]

### Data Cleaning
The `clean_data` function removes any unknown chracters in the provided data and replaces them with the deisgnated unknown chracter of `~`. I'm essentially forfeiting these characters if they ever appear in the testing data, since I likely couldn't get them correct anyway considering the model did not see them during training (unless they appear in the Swahili data, but see my explanation below for that decision).

Inputs:
 - `raw_data`: `string` of raw data read directly from file
 - `known_chars`: `list` of `string` to be included in the data. Everything not in this list will be replaced.

In [85]:
def clean_data(raw_data: str, known_chars: str) -> str:
    cleaned = ""
    for char in raw_data:
        if char not in known_chars:
            cleaned += "~"
        else:
            cleaned += char
    return cleaned

### Data Loading
Load the Kwere training data and split based on the provided ratio. Then load the percentage of the Swahili data requested (see `PARAMS`). Finally, if a test file is provided in `PARAMS`, load the test data.

The validation, Swahili, and test data are all cleaned of unknown chracters. I chose to exclude any chracters found in the Swahili data but not found in the Kwere training data for the sake of staying as true to the Kwere language as possible (in the event Swahili uses a character that Kwere does not).

In [86]:
print("Loading Kwere training data:", end="\n\t")
raw_kwere = open(PARAMS['kwere_train'], 'r').read()
kwere_train_size, kwere_val_size = int(len(raw_kwere)*(1-PARAMS['val_split'])), int(len(raw_kwere)*PARAMS['val_split'])

kwere_train = Dataset(raw_kwere[:kwere_train_size], device)

print("Loading Kwere validation data:", end="\n\t")
cleaned_kwere_val_data = clean_data(raw_kwere[kwere_train_size:], kwere_train.chars)
kwere_val = Dataset(cleaned_kwere_val_data, device)


if PARAMS['swahili_percentage'] > 0:
    print("Loading Swahili data:", end="\n\t")
    raw_swahili = open(PARAMS['swahili'], 'r').read()
    swahili_size = int(len(raw_swahili) * PARAMS['swahili_percentage'])

    cleaned_swahili_data = clean_data(raw_swahili[:swahili_size], kwere_train.chars)
    swahili = Dataset(cleaned_swahili_data, device)


if len(PARAMS['test_data']) > 0:
    print("Loading testing data:", end="\n\t")
    raw_test = open(PARAMS['test_data'], 'r').read()

    cleaned_test_data = clean_data(raw_test, kwere_train.chars)
    test_data = Dataset(cleaned_test_data, device)

Loading Kwere training data:
	422402 characters, 32 unique
Loading Kwere validation data:
	181030 characters, 32 unique
Loading Swahili data:
	392610 characters, 32 unique
Loading testing data:
	61717 characters, 32 unique


### Model Declaration
The model is very similar to those I've used in past challenges: a multilayer LSTM with dropout. I've also added the ability to input a hidden state so the state can be carried between sequences.

In [87]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, dropout):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, input_size)
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers,
            dropout = dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.lstm(embedding, hidden_state)
        output = self.fc(self.dropout(output))
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

### Loss Function
As defined in the challenge requirements, I'm using a cross entropy loss customized to use log base 2 rather than the typical natural log used in PyTorch.

I've also added an assertion making sure no probability distribution sums to more than 1/10,000 plus or minus 1.0

In [88]:
def cross_entropy_loss(outputs, targets):
    batch_size = outputs.shape[0]
    outputs = nn.functional.softmax(outputs, dim=-1)
    
    for prob_dist_sum in torch.sum(outputs, dim=1):
        assert(abs(prob_dist_sum - 1) < 0.0001), "The sum of all probabilities for a character should be 1.0, but got {}".format(prob_dist_sum)
    
    outputs = torch.log2(outputs)
    outputs = outputs[range(batch_size), targets]

    return -torch.mean(outputs)

In [105]:
def get_acc(outputs, targets):
    corrects = 0
    outputs = nn.functional.softmax(outputs, dim=-1)
    
    for idx, output in enumerate(outputs):
        corrects += 1 if torch.argmax(output) == targets[idx] else 0

    return corrects / len(targets)

In [104]:
torch.argmax(torch.Tensor([1,2,3]))

tensor(2)

### Model Declaration
Based on `PARAMS` and the determined `vocab_size` of the train data.

In [89]:
rnn = RNN(
    kwere_train.vocab_size, 
    kwere_train.vocab_size, 
    PARAMS['hidden_size'], 
    PARAMS['num_layers'],
    PARAMS['dropout'],
)

### Optimizer
Using an Adam optimizer, learning rate set in `PARAMS`.

In [90]:
loss_fn = cross_entropy_loss
optimizer = torch.optim.Adam(rnn.parameters(), lr=PARAMS['lr'])

### Learning Rate Modifier
The `set_lr` function is meant to modify the learning rate between pretraining and fine-tuning to avoid overfitting on the Swahili data.

In [91]:
def set_lr(optimizer: torch.optim.Optimizer, lr: int):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### Model to GPU

In [92]:
rnn.to(device)

RNN(
  (embedding): Embedding(32, 32)
  (lstm): LSTM(32, 512, num_layers=4, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=32, bias=True)
)

Function to check for NaNs, used in debugging.

In [93]:
def has_nan(t: torch.Tensor) -> bool:
    if torch.sum(torch.isnan(t)) > 0:
        return True
    return False

### Train Function
Standard train function taking `seq_len` characters at a time. For each character in the series, the LSTM will predict the next character based on the previous character and the character history (represented by the hidden state). 

The hidden state is optionally carried between sequences, so events like a sequence ending mid-word should have no negative effect. I've made this optional because while it could help with cutoff words, I think restarting the hidden state every sequence could also be beneficial as a sort of dropout, in the event a particularly difficult sequence causes the hidden state to be thrown off.

In [106]:
def train(model, criterion, optimizer, data, seq_len):
    ptr = 0
    n = 0
    running_loss = 0
    running_acc = 0
    hidden_state = None
    
    model.train()

    while ptr + seq_len + 1 < len(data):
        input_seq = data[ptr:ptr+seq_len].to(device)
        target_seq = data[ptr+1:ptr+seq_len+1].to(device)

        if hidden_state is not None:
            if has_nan(hidden_state[0]) or has_nan(hidden_state[1]):
                hidden_state = None
        output, hidden_state = model(input_seq, hidden_state if PARAMS['carry_hidden_state'] else None)

        try:
            loss = criterion(torch.squeeze(output), torch.squeeze(target_seq))
            assert(not torch.isnan(loss)), "The loss shouldn't be nan"
            running_loss += loss.item()
            running_acc += get_acc(torch.squeeze(output), torch.squeeze(target_seq))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            n += 1
        except AssertionError as err:
            print("An assertion failed, skipping for now but this shouldn't happen often:\n\t{}".format(err))

        ptr += seq_len

    return running_loss/n, running_acc/n

### Test Function
Standard test function with optional carried hidden state.

In [107]:
def test(model, criterion, data, seq_len):
    ptr = 0
    n = 0
    running_loss = 0
    running_acc = 0
    hidden_state = None
    
    model.eval()

    while ptr + seq_len + 1 < len(data):
        input_seq = data[ptr:ptr+seq_len]
        target_seq = data[ptr+1:ptr+seq_len+1]

        if hidden_state is not None:
            if has_nan(hidden_state[0]) or has_nan(hidden_state[1]):
                hidden_state = None
        output, hidden_state = model(input_seq, hidden_state if PARAMS['carry_hidden_state'] else None)

        try:
            loss = criterion(torch.squeeze(output), torch.squeeze(target_seq))
            assert(not torch.isnan(loss)), "The loss shouldn't be nan"
            running_loss += loss.item()
            running_acc += get_acc(torch.squeeze(output), torch.squeeze(target_seq))
            
            n += 1
        except AssertionError as err:
            print("An assertion failed, skipping for now but this shouldn't happen often:\n\t{}".format(err))

        ptr += seq_len
        
    return running_loss/n, running_acc/n

### Loss Function Verification
Based on the equation for cross entropy, a randomized model's loss should on average be $log_2(vocab\_size)$.

This number should also be the target to verify that the model is learning. Any loss lower than this value has learned a non-zero amount.

In [96]:
print("Vocab size is {}, so cross entropy with no training should be approximately {}".format(kwere_train.vocab_size, math.log(kwere_train.vocab_size, 2)))
print("Untrained loss:", end=" ")
print(test(rnn, loss_fn, kwere_val, PARAMS['seq_len']))

Vocab size is 32, so cross entropy with no training should be approximately 5.0
Untrained loss: 5.015613420233542


### Pretrain

In [97]:
if PARAMS['swahili_percentage'] > 0:
    set_lr(optimizer, PARAMS['pretrain_lr'])
    
    for epoch in range(0, PARAMS['pretrain_epochs']):
        print("-"*3 + " Pretrain Epoch {} ".format(epoch+1) + "-"*17)

        print("\tPretrain Loss:", end=" ")
        pretrain_loss = train(rnn, loss_fn, optimizer, swahili, PARAMS['seq_len'])
        print(pretrain_loss)
        
        if is_logging:
            neptune.log_metric("Pretrain Loss", pretrain_loss)

--- Pretrain Epoch 1 -----------------
	Pretrain Loss: 3.93833114599726
--- Pretrain Epoch 2 -----------------
	Pretrain Loss: 3.266909222086524
--- Pretrain Epoch 3 -----------------
	Pretrain Loss: 3.0710979352331464
--- Pretrain Epoch 4 -----------------
	Pretrain Loss: 2.9373063609858225
--- Pretrain Epoch 5 -----------------
	Pretrain Loss: 2.8453641997780768


### Training

In [110]:
set_lr(optimizer, PARAMS['lr'])

for epoch in range(0, PARAMS['epochs']):
    print("-"*3 + " Epoch {} ".format(epoch+1) + "-"*25)
    
    print("\tTrain Loss:", end=" ")
    train_loss, train_acc = train(rnn, loss_fn, optimizer, kwere_train, PARAMS['seq_len'])
    print("{:.3f}".format(train_loss), end="\t")
    print("\t|\tTrain Accuracy:", end=" ")
    print("{:.2f}%".format(train_acc*100))
    if is_logging:
        neptune.log_metric("Train Loss", train_loss)
    
    print("\tValidation Loss:", end=" ")
    val_loss, val_acc = test(rnn, loss_fn, kwere_val, PARAMS['seq_len'])
    print("{:.3f}".format(val_loss), end="\t")
    print("\t|\tValidation Accuracy:", end=" ")
    print("{:.2f}%".format(val_acc*100))
    if is_logging:
        neptune.log_metric("Validation Loss", val_loss)

--- Epoch 14 -------------------------
	Train Loss: 1.233		|	Train Accuracy: 71.93%
	Validation Loss: 1.401		|	Validation Accuracy: 69.65%
--- Epoch 15 -------------------------
	Train Loss: 1.222		|	Train Accuracy: 72.14%
	Validation Loss: 1.404		|	Validation Accuracy: 69.69%
--- Epoch 16 -------------------------
	Train Loss: 

KeyboardInterrupt: 

### Testing

In [111]:
if len(PARAMS['test_data']) > 0:
    print("Test Loss:", end=" ")
    test_loss, test_acc = train(rnn, loss_fn, optimizer, test_data, PARAMS['seq_len'])
    print("{:.3f}".format(test_loss), end="\t")
    print("\t|\tTest Accuracy:", end=" ")
    print("{:.2f}%".format(test_acc*100))

Test Loss: 1.478		|	Test Accuracy: 67.80%
